# Project MSI 2 – Water chemical potential
by Cristina Prat

Download all the required files for the project from this [link](http://pub.htmd.org/CXCL12-confAnalysis/)

### Objective

We have to create a python module which given a set of molecular dynamics trajectories computes the volumetric map of the chemical potential for water.

Assuming the simulations are in equilibrium, which is a reasonable assumption for water molecules, we have to measure the average water occupancy p(x,y,z) from the oxigen atom of water molecules on a 3D regular grid of 1A for each dimension.

### Getting started

First we import the modules, molecules and its trajectories we are going to need for the project:

In [1]:
from htmd import *
from htmd.molecule.util import maxDistance, writeVoxels
htmd.config(viewer='vmd')

New HTMD version (1.0.22) is available. You are currently on (1.0.16). Use 'conda update htmd' to update to the new version.


Create a list of simulations for all the folders where we have trajectories:

In [2]:
sims = simlist(glob('./CXCL12-confAnalysis/*/'), glob('./CXCL12-confAnalysis/*/structure.pdb'))

Creating simlist: 100% (10/10) [###################################] eta 00:01 -


We define a function where for each molecule we wrap the coordinates back into the unit cell. Molecules will remain continuous, so may escape the bounds of the prinary unit cell. After that, the molecule is aligned to a reference structure:

In [3]:
def unit_cell(s):
    molTraj = Molecule(s)
    molTraj.wrap('protein')
    molTraj.align('protein')
    return molTraj

As we need all the trajectories for this molecule, we apply the function "unit_cell" for each of them in order to get a list of trajectories, a way to select each trajectory easily:

In [4]:
trajectories = []
for s in sims:
    trajectories.append(unit_cell(s))

At this point, by maxDistance we can get the maximum distance in angstrom of all the atoms of the molecules from its center. Therefore, as we have all the trajectories in a list, we can create another list where we put all the maximum distances and get the maximum of them:

In [5]:
D = []
for molTraj in trajectories:
    D.append(maxDistance(molTraj, 'all'))

In [6]:
D_max = int(np.max(np.round(D)))
D_max

53

### Reposition of the molecule

We need to move all molecules in order to avoid negative coordinates by a given vector which in that case is the value of the maximum distance to the center (D_max). Molecule.moveBy is used to translate the molecule and center it on [D_max, D_max, D_max]:

In [7]:
for molTraj in trajectories:
    molTraj.moveBy([D_max,D_max,D_max])

### Working with water duplicate object

We create a new array of given shape and type filled with zeros. The dimension of this matrix is given by the double value of D_max with some extra points. We add some more points to take account waters along the trajectories because we have obtained the D_max only by the first frame. 

The matrix is used to count the number of water atoms on a 3D regular grid of 1A for each dimension:

In [8]:
dim_matrix = (D_max * 2) + 10
matrix = np.zeros((dim_matrix, dim_matrix, dim_matrix))

Now, we have to create a water duplicate object in order to simplify the code to measure the average water occupancy p(x,y,z) from the oxigen atom of water molecules on a 3D regular grid of 1A for each dimension. This is done by each trajectory.

Molecule.filter is used to select water parts of the molecule and clean/remove all the rest.

Let's iterate by trajectories, frames and atoms in the water object. This loop takes a while:

In [9]:
for molTraj in trajectories:
    water = molTraj.copy()
    water.filter('water and name OH2')
    for frame in range(len(water.step)):
        try:
            for atom in range(len(water.coords)):
                coords = water.coords[atom, :, frame]
                matrix[int(round(coords[0]))][int(round(coords[1]))][int(round(coords[2]))] += 1
        except IndexError:
            pass

The chemical potential is obtained from the occupancy as G(x,y,z)= - kB T log(p(x,y,z)). But, before calculate it, we need to normalize our matrix by the number of frames, coords and trajectories:

In [10]:
norm_matrix = matrix / (len(water.step) * len(water.coords) * len(sims))

Let's define the constants for calculate the chemical potential:

In [11]:
kb = 0.001987191 # kcal/mol/K
T = 298 # K

In [12]:
occupancy = - kb * T * np.log(norm_matrix + 1e-20)

### Save a volumetric file

We need to define the different parameters for 'writeVoxel':
- vecMin: 3D vector denoting the minimal corner of the grid
- vecMax: 3D vector denoting the maximal corner of the grid
- vecRes: 3D vector denoting the resolution of the grid in each dimension

In [13]:
vecMin = np.array([0,0,0])
vecMax = np.array([dim_matrix, dim_matrix, dim_matrix])
vecRes = np.array([1,1,1])

We use the function 'writeVoxel' of VMD to save a volumetric file (.cube):

In [14]:
writeVoxels(occupancy, 'box.cube', vecMin, vecMax, vecRes)

It can then visualized on top of the PDB of the molecule as an isosurface on VMD. So, when we open the following command, we have to load data into molecule and browse the 'box.cube' generated.

We can select the 'Drawing Method' called 'Isosurface' which computes and draws a surface within a volumetric data field, on a 3D surface corresponding to points with a single scalar value.

In [15]:
molTraj.view(sel='protein', style='NewCartoon')

The following image shows the 'structure.pdb', the 'Isosurface' 'box.cube' calculated and also the 'VolumeSlice'.
- The 'Isosurface' representation is drawed given a isovalue of 7.5 and step 2.
- The 'VolumeSlice' representation draws a texture mapped two-dimensional slice from a volumetric data set already loaded into VMD

<img src="project_2.png" alt="Drawing" style="width: 1000px;"/>